The cell below prints out VaR and CVaR estimates for Normal and Student's t distribution at confidence level 0.95 and 0.99 using the SGLD algorithm. 

It runs for about 7min.

In [ ]:
import numpy as np
import scipy.stats as st

###############################################################################
# SGLD FUNCTION: RUNS THE ALGORITHM AND RETURNS SAMPLES OF THETA
###############################################################################

def sgld_var_cvar(dist_sampler, q, N, gamma, beta, lam, collect, random_seed=None):
    """
    Perform SGLD to minimize V(theta) = E[ theta + (1/(1-q)) * (X - theta)_+ ] + gamma * theta^2 .
    
    Input:
        dist_sampler: a function() -> sample of X from the chosen distribution.
        q: confidence level for VaR/CVaR.
        N: total number of iterations.
        gamma: positive constant in the theta^2 penalization term.
        beta: (positive) inverse-temperature parameter used in the SGLD noise.
        lam: (positive) step size.
        collect: number of samples to collect at the end of the iterations.
        random_seed: to fix the seed for reproducibility.
    
    Returns:
        final_thetas: array of collected samples of theta.
        final_Xs:     corresponding X draws from the tail of the chain.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    N = int(N)  # Ensure N is an integer

    theta = 0.0   # initial value, theta_0 = 0
    thetas = np.zeros(N)
    Xs = np.zeros(N)
    
    # Main SGLD loop
    for n in range(N):

        # Sample X_{n+1} from the chosen distribution
        x = dist_sampler()
        
        # Compute the stochastic gradient H:
        # H(theta, x) = 1 + 2 * gamma * theta    if x < theta,
        #             = 1 - 1/(1-q) + 2 * gamma * theta  if x >= theta.
        if x < theta:
            H = 1.0 + 2 * gamma * theta
        else:
            H = 1.0 - 1.0/(1.0 - q) + 2 * gamma * theta
        
        # Update rule:
        # theta_{n+1} = theta_n - lam * H + sqrt(2*lam/beta)*N(0,1)
        theta = theta - lam * H + np.random.normal(loc=0.0, scale=np.sqrt(2.0 * lam / beta))
        
        thetas[n] = theta
        Xs[n] = x

    # Return the last 'collect' samples after burn_in period
    final_thetas = thetas[-collect:]
    final_Xs     = Xs[-collect:]
    
    return final_thetas, final_Xs

###############################################################################
# FUNCTION TO ESTIMATE VaR AND CVaR FROM THE SGLD ALGORITHM
###############################################################################

def estimate_var_cvar(thetas, xs, q, gamma):
    """
    Given arrays of theta and x, estimate:
        VaR_SGLD = average of theta,
        CVaR_SGLD = average of [ theta + (1/(1-q)) * (x - theta)_+ ] + gamma * theta^2.
    """
    var_est = np.mean(thetas)

    cvar_terms = thetas + (1.0 / (1.0 - q)) * np.maximum(0.0, xs - thetas)
    cvar_est = np.mean(cvar_terms + gamma * thetas**2)

    return var_est, cvar_est


###############################################################################
# EXPERIMENTS: PRINT RESULTS FOR THE NORMAL AND STUDENT'S T CASES
###############################################################################

def run_normal_experiments(q):
    """
    Run SGLD experiments for the Normal distribution case using independent simulations.
    
    For each parameter set (mu, sigma) from the list [(0,1), (1,2), (3,5)],
    we run 5 independent SGLD simulations, each with 10^6 iterations.
    From each simulation, we take the last 2000 theta's and corresponding x's.
    The VaR and CVaR for the given confidence level q (via SGLD) are computed
    and then averaged over the simulations.
    """
    normal_params = [(0, 1), (1, 2), (3, 5)]
    num_simulations = 5  # Total independent simulations
    N = int(1e6)  # Number of iterations per simulation
    gamma = 1e-8
    beta = 1e8
    lam = 1e-4
    collect = 2000  # Number of samples to collect
    
    for mu, sigma in normal_params:
        vars = []
        cvars = []
        
        # Create a sampler function for Normal(mu, sigma)
        def make_normal_sampler(mu, sigma):
            return lambda: np.random.normal(loc=mu, scale=sigma)
        
        dist_sampler = make_normal_sampler(mu, sigma)
        
        # For each independent simulation, we run SGLD
        for i in range(num_simulations):
            # Use a different seed in each simulation for reproducibility.
            seed = 42 + i  
            theta, x = sgld_var_cvar(dist_sampler, q, N, gamma, beta, lam, collect, random_seed=seed)
            
            var, cvar = estimate_var_cvar(theta, x, q, gamma)
            vars.append(var)
            cvars.append(cvar)
        
        # Average over 5 independent simulations.
        var_avg = np.mean(vars)
        var_std = np.std(vars)
        cvar_avg = np.mean(cvars)
        cvar_std = np.std(cvars)
        
        normal_star = {
            (0, 1): (1.645, 2.062, 2.326, 2.677),
            (1, 2): (4.290, 5.124, 5.653, 6.335),
            (3, 5): (11.224, 13.311, 14.632, 16.337)
        }
        v_star_95, c_star_95, v_star_99, c_star_99 = normal_star[(mu, sigma)]

        if q == 0.95:
            print(f"N({mu}, {sigma}) --> "
                f"VaR* = {v_star_95:.3f}, CVaR* = {c_star_95:.3f} | "
                f"VaR_SGLD = {var_avg:.3f} ({var_std:.2f}), "
                f"CVaR_SGLD = {cvar_avg:.3f} ({cvar_std:.4f})")
        else:
            print(f"N({mu}, {sigma}) --> "
                f"VaR* = {v_star_99:.3f}, CVaR* = {c_star_99:.3f} | "
                f"VaR_SGLD = {var_avg:.3f} ({var_std:.2f}), "
                f"CVaR_SGLD = {cvar_avg:.3f} ({cvar_std:.4f})")
   
        
def run_student_t_experiments(q):
    """
    Run SGLD experiments for the Student's t distribution case using independent simulations.
    
    For each degrees of freedom value in [10, 7, 3],
    we run 5 independent SGLD simulations, each with 10^6 iterations.
    From each simulation, we take the last 2000 theta's and corresponding x's.
    The VaR and CVaR for the given confidence level q (via SGLD) are computed
    and then averaged over the simulations.
    """
    t_params = [10, 7, 3]
    num_simulations = 5  # Total independent simulations
    N = int(1e6)  # Number of iterations per simulation
    gamma = 1e-8
    beta = 1e8
    lam = 1e-4
    collect = 2000  # Number of samples to collect
    
    for df in t_params:
        vars = []
        cvars = []
        
        # Create a sampler function for Student's t with df degrees of freedom.
        def make_t_sampler(df):
            return lambda: st.t.rvs(df)
        
        dist_sampler = make_t_sampler(df)
        
        # Run the independent simulations.
        for i in range(num_simulations):
            seed = 42 + i  # Different seed for each simulation run.
            theta, x = sgld_var_cvar(dist_sampler, q, N, gamma, beta, lam, collect, random_seed=seed)
            
            var, cvar = estimate_var_cvar(theta, x, q, gamma=gamma)
            vars.append(var)
            cvars.append(cvar)
        
        # Average over 5 independent simulations.
        var_avg = np.mean(vars)
        var_std = np.std(vars)
        cvar_avg = np.mean(cvars)
        cvar_std = np.std(cvars)
        
        t_star = {
            10: (1.812, 2.416, 2.764, 3.357),
            7:  (1.895, 2.595, 2.998, 3.757),
            3:  (2.353, 3.876, 4.541, 6.968)
        }
        v_star_95, c_star_95, v_star_99, c_star_99 = t_star[df]

        if q == 0.95:
            print(f"t({df}) --> "
                f"VaR* = {v_star_95:.3f}, CVaR* = {c_star_95:.3f} | "
                f"VaR_SGLD = {var_avg:.3f} ({var_std:.2f}), "
                f"CVaR_SGLD = {cvar_avg:.3f} ({cvar_std:.4f})")
        else:
            print(f"t({df}) --> "
                f"VaR* = {v_star_99:.3f}, CVaR* = {c_star_99:.3f} | "
                f"VaR_SGLD = {var_avg:.3f} ({var_std:.2f}), "
                f"CVaR_SGLD = {cvar_avg:.3f} ({cvar_std:.4f})")

        
###############################################################################
# RUN EXPERIMENTS
###############################################################################

print("VaR and CVaR estimates for Normal and Student's t distributions")
print("averaged over 5 independent SGLD simulations, each using the last 2000 samples to average over:\n")

print("Normal Distribution:\n")
print("Confidence level 0.95:")
normal_95 = run_normal_experiments(0.95)
print("\n")
print("Confidence level 0.99:")
normal_99 = run_normal_experiments(0.99)

print("\n")

print("Student's t Distribution:\n")
print("Confidence level 0.95:")
student_95 = run_student_t_experiments(0.95)
print("\n")
print("Confidence level 0.99:")
student_99 = run_student_t_experiments(0.99)

VaR and CVaR estimates for Normal and Student's t distributions
averaged over 5 independent SGLD simulations, each using the last 2000 samples to average over:

Normal Distribution:

Confidence level 0.95:
N(0, 1) --> VaR* = 1.645, CVaR* = 2.062 | VaR_SGLD = 1.645 (0.02), CVaR_SGLD = 2.088 (0.0344)
N(1, 2) --> VaR* = 4.290, CVaR* = 5.124 | VaR_SGLD = 4.285 (0.04), CVaR_SGLD = 5.178 (0.0678)
N(3, 5) --> VaR* = 11.224, CVaR* = 13.311 | VaR_SGLD = 11.209 (0.06), CVaR_SGLD = 13.446 (0.1685)


Confidence level 0.99:
N(0, 1) --> VaR* = 2.326, CVaR* = 2.677 | VaR_SGLD = 2.324 (0.04), CVaR_SGLD = 2.706 (0.0529)
N(1, 2) --> VaR* = 5.653, CVaR* = 6.335 | VaR_SGLD = 5.651 (0.06), CVaR_SGLD = 6.418 (0.1074)
N(3, 5) --> VaR* = 14.632, CVaR* = 16.337 | VaR_SGLD = 14.628 (0.11), CVaR_SGLD = 16.555 (0.2747)


Student's t Distribution:

Confidence level 0.95:
t(10) --> VaR* = 1.812, CVaR* = 2.416 | VaR_SGLD = 1.817 (0.00), CVaR_SGLD = 2.462 (0.1346)
t(7) --> VaR* = 1.895, CVaR* = 2.595 | VaR_SGLD = 1.9